<a href="https://colab.research.google.com/github/brendan123/Semantic-Segmentation/blob/milestone-3/Milestone_3_SemanticSegementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Milestone 3: Perform Baseline Semantic Segmentation

## Install Dependencies

In [1]:
# Seed repo Python dependencies
! pip install patchify
! pip install segmentation-models
! pip install -q keras # Keras

#kaggle
!pip install -q kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 50 kB 5.9 MB/s 


## Downloading Dataset from Kaggle

In [2]:
from google.colab import files
files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 700 ~/.kaggle/kaggle.json
! kaggle datasets download -d humansintheloop/semantic-segmentation-of-aerial-imagery
! unzip semantic-segmentation-of-aerial-imagery.zip
! rm semantic-segmentation-of-aerial-imagery.zip
! rm kaggle.json # Delete kaggle.json

Saving kaggle.json to kaggle.json
 57% 17.0M/29.6M [00:00<00:00, 81.2MB/s]
100% 29.6M/29.6M [00:00<00:00, 113MB/s] 
Archive:  semantic-segmentation-of-aerial-imagery.zip
  inflating: Semantic segmentation dataset/Tile 1/images/image_part_001.jpg  
  inflating: Semantic segmentation dataset/Tile 1/images/image_part_002.jpg  
  inflating: Semantic segmentation dataset/Tile 1/images/image_part_003.jpg  
  inflating: Semantic segmentation dataset/Tile 1/images/image_part_004.jpg  
  inflating: Semantic segmentation dataset/Tile 1/images/image_part_005.jpg  
  inflating: Semantic segmentation dataset/Tile 1/images/image_part_006.jpg  
  inflating: Semantic segmentation dataset/Tile 1/images/image_part_007.jpg  
  inflating: Semantic segmentation dataset/Tile 1/images/image_part_008.jpg  
  inflating: Semantic segmentation dataset/Tile 1/images/image_part_009.jpg  
  inflating: Semantic segmentation dataset/Tile 1/masks/image_part_001.png  
  inflating: Semantic segmentation dataset/Tile 1/m

## Run Seed Repo Code

Modified from Dr. Sreenivas Bhattiprolu's original repo found [here](https://github.com/bnsreenu/python_for_microscopists/tree/master/228_semantic_segmentation_of_aerial_imagery_using_unet).

Changes made:


* We were getting a problem where our images and masks were not matching in the array, so we sorted the images and masks by name when walking the directory.
* Uncommented model save line.
* Removed version with pre-trained weights as it was less performant.
* Removed unnecessary blocks from original notebook (such as average IoU calculation).
* Modified "Predict on a few images" to show 10 images.
* Added block for precision/recall curve.
* Removed training and validation IoU over epochs.
* Changed epochs to 50.
* Added metrics Precision and Recall.
* Added NNI parameters for optimization.


## NNI HPO

Using HyperBand to minimize loss.

In [10]:
# Download model.py from out GitHub repo
! wget -O model.py https://raw.githubusercontent.com/brendan123/Semantic-Segmentation/milestone-3/model.py

# NNI
! pip install nni # install nni
! mkdir -p nni_repo
! git clone https://github.com/microsoft/nni.git nni_repo/nni # clone NNI's offical repo to get examples

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 56.0 MB 1.3 MB/s 
     |████████████████████████████████| 51 kB 8.3 MB/s 
     |████████████████████████████████| 106 kB 85.3 MB/s 
     |████████████████████████████████| 130 kB 90.4 MB/s 
     |████████████████████████████████| 127 kB 79.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


Cloning into 'nni_repo/nni'...
remote: Enumerating objects: 58988, done.
remote: Counting objects: 100% (305/305), done.
remote: Compressing objects: 100% (265/265), done.
remote: Total 58988 (delta 100), reused 165 (delta 33), pack-reused 58683
Receiving objects: 100% (58988/58988), 121.95 MiB | 38.63 MiB/s, done.
Resolving deltas: 100% (35352/35352), done.


Set up NNI Experiment

In [79]:
from nni.experiment import Experiment
experiment = Experiment('local')

search_space = {
    'batch_size': {'_type': 'randint', '_value': [1, 32]},
    'learning_rate': {'_type': 'loguniform', '_value': [0.0001, 0.1]},
    'activation_type': {'_type': 'choice', '_value': ['relu', 'tanh', 'swish', None]}
}

experiment.config.trial_command = 'python model.py'
experiment.config.trial_code_directory = '.'
experiment.config.search_space = search_space
experiment.config.tuner.name = 'HyperBand'
experiment.config.tuner.class_args['optimize_mode'] = 'minimize'
experiment.config.max_trial_number = 72
experiment.config.trial_concurrency = 4

Download and install Ngrok. Get authtoken from user.

In [36]:
# Ngrok
! wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip # download ngrok and unzip it
! unzip ngrok-stable-linux-amd64.zip && rm ngrok-stable-linux-amd64.zip

# This is my authtoken please don't take it
! ./ngrok authtoken 26JQrkyWKQCjAu3oESdi9hlbctZ_4VPqxWtWrM7YaqYxzUsoW

--2022-11-12 00:38:07--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.237.133.81, 52.202.168.65, 18.205.222.128, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.237.133.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  39.4MB/s    in 0.3s    

2022-11-12 00:38:08 (39.4 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ngrok                   
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


Kill the old Ngrok Instance and start a new one for port 5000

In [80]:
# Kills old ngrok instance running on 5000 (if running)
! kill $(lsof -t -i:5000)
get_ipython().system_raw('./ngrok http 5000 &')

kill: usage: kill [-s sigspec | -n signum | -sigspec] pid | jobspec ... or kill -l [sigspec]


Get the Ngrok URL

In [81]:
! curl -s http://localhost:4040/api/tunnels

{"tunnels":[{"name":"command_line","uri":"/api/tunnels/command_line","public_url":"https://9d7a-35-184-13-121.ngrok.io","proto":"https","config":{"addr":"http://localhost:5000","inspect":true},"metrics":{"conns":{"count":0,"gauge":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0},"http":{"count":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0}}},{"name":"command_line (http)","uri":"/api/tunnels/command_line%20%28http%29","public_url":"http://9d7a-35-184-13-121.ngrok.io","proto":"http","config":{"addr":"http://localhost:5000","inspect":true},"metrics":{"conns":{"count":0,"gauge":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0},"http":{"count":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0}}}],"uri":"/api/tunnels"}


Delete old results (be careful with this)

In [82]:
# Delete old results
! rm -rf models

Begin the experiment!

In [83]:
experiment.run(5000)

[2022-11-12 02:57:22] Creating experiment, Experiment ID: te0bfdzg


INFO:nni.experiment:Creating experiment, Experiment ID: ${CYAN}te0bfdzg


[2022-11-12 02:57:22] Starting web server...


INFO:nni.experiment:Starting web server...


[2022-11-12 02:57:23] Setting up...


...

[2022-11-12 05:44:52] WARNING: KeyboardInterrupt detected


[2022-11-12 05:44:52] Stopping experiment, please wait...


INFO:nni.experiment:Stopping experiment, please wait...


[2022-11-12 05:44:52] Experiment stopped


INFO:nni.experiment:Experiment stopped


## Results

Reults for all trials saved in models directory and the best is displayed in our docs folder.